In [149]:
import sys
import os

project_root = os.path.abspath('..')
if project_root not in sys.path:
    sys.path.append(project_root)

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [166]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns


from src.features.build_features import calcular_kaplan_meier

COLOR_PALETTE = [
    "#05a298",
    "#0070bc",
    "#d06b6d",
    "#a8e667",
    "#57832e",
    "#56ebd3",
    "#0a4f4e",
    "#9ecbf4",
]

REEMPLAZO_NOMBRE_TRASPLANTES = {
    "33.51 - TRASPLANTE PULMON.UNILATERAL": "Trasplante Pulmón Unilateral",
    "37.51 - TRASPLANTE CARDIACO": "Trasplante Cardiaco",
    "33.52 - TRASPLANTE PULMON.BILATERAL": "Trasplante Pulmón Bilateral",
}


def find_nearest_index(df, column_name, target_value):
    # Calculate the absolute difference
    df["abs_diff"] = abs(df[column_name] - target_value)

    # Find the index of the row with the minimum absolute difference
    nearest_index = df["abs_diff"].idxmin()

    return nearest_index


sns.set_style("ticks")
sns.set_palette(COLOR_PALETTE)
plt.rcParams["figure.figsize"] = (12, 6)

## Análisis de Sobrevivencia

En este cuadernillo se quiere realizar el análisis de sobrevivencia de pacientes con
trasplante cardiaco en el Instituto Nacional del Tórax. Para este efecto, se utilizará
la base de datos de GRD de pacientes que se hayan realizado un trasplante cardiaco. La base
sigue a pacientes entre 2011 hasta el 16/08/2023 del año 2022.

Las respuestas que se quieren seguir son las siguientes:

- Luego de 2 años del trasplante, ¿Cuál es la probabilidad de que el paciente siga vivo?
- Luego de 10 años del trasplante, ¿Cuál es la probabilidad de que el paciente siga vivo?

Además, se quiere saber cómo es la curva de sobrevivencia para distintos rangos etarios de
pacientes.


In [151]:
df = pd.read_excel("../data/raw/Trasplantes 2011-2022 16082023.xlsx", sheet_name="bdgrd")

In [152]:
fecha_corte = pd.to_datetime("16/08/2023", dayfirst=True)
df["rango_etario"] = pd.cut(df["Edad en años"], bins=range(0, 121, 10))
df["fecha_tope"] = df["Fecha fallecimiento"].fillna(fecha_corte)
df["duracion_anios"] = (df["fecha_tope"] - df["1ra Intervención Fecha"]).dt.days / 365
df["muerto"] = df["Fecha fallecimiento"].notna()

In [153]:
RENAME_COLUMNAS = {
    "1ra Intervención Fecha": "fecha_trasplante",
    "Fecha fallecimiento": "fecha_muerte",
    "rango_etario": "rango_etario",
    "Proced 01 Principal (cod+des)": "tipo_trasplante",
    "Diag 01 Principal (cod+des)": "diagnostico",
    "fecha_tope": "fecha_tope",
    "duracion_anios": "duracion_anios",
    "muerto": "muerto",
}

In [154]:
df_sobrevivencia = df[RENAME_COLUMNAS.keys()]
df_sobrevivencia = df_sobrevivencia.rename(columns=RENAME_COLUMNAS)
df_sobrevivencia["tipo_trasplante"] = df_sobrevivencia["tipo_trasplante"].replace(
    REEMPLAZO_NOMBRE_TRASPLANTES
)

In [176]:
display(df_sobrevivencia.head(2))

,fecha_trasplante,fecha_muerte,rango_etario,tipo_trasplante,diagnostico,fecha_tope,duracion_anios,muerto
0,2011-01-07,2012-09-14,"(50, 60]",Trasplante Pulmón Unilateral,J84.1 - Otras enfermedades pulmonares intersti...,2012-09-14,1.687671,True
1,2011-01-18,2012-08-02,"(50, 60]",Trasplante Pulmón Unilateral,J84.1 - Otras enfermedades pulmonares intersti...,2012-08-02,1.539726,True
2,2011-02-24,NaT,"(50, 60]",Trasplante Cardiaco,I42.0 - Cardiomiopatía dilatada,2023-08-16,12.482192,False
3,2011-02-24,2022-10-14,"(40, 50]",Trasplante Pulmón Unilateral,J84.1 - Otras enfermedades pulmonares intersti...,2022-10-14,11.643836,True
4,2011-04-12,NaT,"(50, 60]",Trasplante Pulmón Unilateral,J84.1 - Otras enfermedades pulmonares intersti...,2023-08-16,12.353425,False


In [175]:
SOBREVIDAS_A_ANALIZAR = [5, 10, 12]
MARGEN_TEXTO_ARRIBA = 0.012
MARGEN_TEXTO_DERECHA = 0.15

# fig, axis = plt.subplots(1, 1, figsize=(12, 6))
# for tipo_trasplante in sorted(df_sobrevivencia["tipo_trasplante"].unique()):
#     df_trasplante = df_sobrevivencia.query("tipo_trasplante == @tipo_trasplante")
#     resultado_kaplan = calcular_kaplan_meier(df_trasplante, "duracion_anios", "muerto")

#     sns.lineplot(
#         resultado_kaplan,
#         x="tiempo",
#         y="probabilidad_sobrevivencia",
#         label=f"{tipo_trasplante} - n = {df_trasplante.shape[0]}",
#     )
#     plt.fill_between(
#         resultado_kaplan["tiempo"],
#         resultado_kaplan["limite_confianza_bajo"],
#         resultado_kaplan["limite_confianza_alto"],
#         alpha=0.05,
#     )

#     for sobrevida in SOBREVIDAS_A_ANALIZAR:
#         duracion_a_sobrevida_anios = find_nearest_index(resultado_kaplan, "tiempo", sobrevida)
#         probabilidad_a_sobrevida_anios = resultado_kaplan.iloc[duracion_a_sobrevida_anios, 1]
#         plt.plot(sobrevida, probabilidad_a_sobrevida_anios, color="black", marker="o", markersize=3)
#         plt.text(
#             sobrevida + MARGEN_TEXTO_DERECHA,
#             probabilidad_a_sobrevida_anios + MARGEN_TEXTO_ARRIBA,
#             f"{(probabilidad_a_sobrevida_anios * 100):.1f}%",
#             fontsize=12,
#         )


# # for sobrevida in SOBREVIDAS_A_ANALIZAR:
# #     plt.axvline(sobrevida, color="gray", linewidth=1, linestyle="--")

# plt.ylim(0, 1)
# plt.xlim(left=0)
# plt.title("Sobrevida a Largo Plazo en Trasplantes Cardíacos y de Pulmón Unilateral/Bilateral")
# plt.xlabel("Tiempo (Años)")
# plt.ylabel("Sobrevivencia (%)")
# plt.show()

In [157]:
sns.set_style()